In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [53]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
data=pd.read_csv('zomato_banglore.csv')

In [3]:
data.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [4]:
#remove columns
copy=data.drop(['url','online_order', 'book_table','phone','dish_liked','votes','menu_item'],axis=1)

In [5]:
#changing column names 
copy.rename(columns={'listed_in(type)':'Type','listed_in(city)':'City','approx_cost(for two people)':'Cost'},inplace=True)

In [6]:
#check for duplicates
print(copy.duplicated(['name','address']).sum())
copy.drop_duplicates(['name','address'],inplace=True,keep='first')

39218


In [7]:
copy.shape[0]

12499

In [8]:
#checking and deleting nan values
copy.isna().sum()
copy.dropna(inplace=True)

In [9]:
#removing new restaurants and restaurants with no ratings
copy = copy.loc[copy.rate !='NEW']
copy= copy.loc[copy.rate !='-'].reset_index(drop=True)

In [10]:
for i in range(0,copy.shape[0]):
        rating=copy['rate'][i].replace('/5','')
        copy['rate'][i]=float(rating)

In [11]:
copy.keys()

Index(['address', 'name', 'rate', 'location', 'rest_type', 'cuisines', 'Cost',
       'reviews_list', 'Type', 'City'],
      dtype='object')

In [12]:
restaurants=list(copy['name'].unique())

In [13]:
len(restaurants)

6465

In [14]:
copy.shape[0]

9244

In [15]:
copy['Mean Rating']=0.0

In [16]:
mean_ratings=[]
for i in range(len(restaurants)):
    mean_ratings.append((copy['rate'][copy['name']==restaurants[i]]).mean())
    copy['Mean Rating'][copy['name']==restaurants[i]]=mean_ratings[i]

<ipython-input-16-21342abeae49>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy['Mean Rating'][copy['name']==restaurants[i]]=mean_ratings[i]


In [17]:
copy.keys()

Index(['address', 'name', 'rate', 'location', 'rest_type', 'cuisines', 'Cost',
       'reviews_list', 'Type', 'City', 'Mean Rating'],
      dtype='object')

In [18]:
#review list cleaning
len(copy['reviews_list'][0])

3955

In [19]:
import re
copy["reviews_list"] = copy["reviews_list"].str.lower()

## Removal of Puctuations
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

copy["reviews_list"] = copy["reviews_list"].apply(lambda text: remove_punctuation(text))

## Removal of Stopwords
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

copy["reviews_list"] = copy["reviews_list"].apply(lambda text: remove_stopwords(text))

## Removal of URLS
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

copy["reviews_list"] = copy["reviews_list"].apply(lambda text: remove_urls(text))

copy[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
1331,rated 30 ratedn original glazed donuts reason ...,"Desserts, Beverages"
7550,,Mangalorean
1325,rated 30 ratedn average thats ordered lucknowi...,"Biryani, North Indian, Rolls"
295,rated 40 ratedn one best dishes iãx83ãx83ãx...,"Seafood, Mangalorean"
6992,rated 50 ratedn lovely food affordable price t...,"Fast Food, Burger, Pizza, Sandwich"


In [21]:
copy.duplicated(['name']).sum()

2779

In [22]:
#restaurant branches
res_counts=[]
for i in restaurants:
    counts=copy[copy['name']==i].shape[0]
    res_counts.append(counts)

In [24]:
#combining the reviews of same restaurants
review_list=[]
for i in restaurants:
    review=""
    for j in range(0,copy.shape[0]):
        if(i==copy['name'][j]):
            review+=copy['reviews_list'][j]
    review_list.append(review)        

In [25]:
len(review_list)

6465

In [26]:
all_res=copy.drop_duplicates(['name'])    

In [27]:
all_res.head()

,address,name,rate,location,rest_type,cuisines,Cost,reviews_list,Type,City,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800,rated 40 ratedn beautiful place dine inthe int...,Buffet,Banashankari,4.15
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800,rated 40 ratedn dinner family turned good choo...,Buffet,Banashankari,4.10
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,3.8,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800,rated 30 ratedn ambience good enough pocket fr...,Buffet,Banashankari,3.80
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,3.7,Banashankari,Quick Bites,"South Indian, North Indian",300,rated 40 ratedn great food proper karnataka st...,Buffet,Banashankari,3.70
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,3.8,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600,rated 40 ratedn good restaurant neighbourhood ...,Buffet,Banashankari,3.80


In [28]:
for i in range(len(restaurants)):
    all_res['reviews_list'][all_res['name']==restaurants[i]]=review_list[i]

<ipython-input-28-b28e10ab05c9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_res['reviews_list'][all_res['name']==restaurants[i]]=review_list[i]
c:\users\jaswanth reddy\appdata\local\programs\python\python39\lib\site-packages\pandas\core\series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


In [29]:
all_res.reset_index(inplace=True)

In [30]:
cuisines=[]
for i in range(0,all_res.shape[0]):
    temp=all_res['cuisines'][i].split(',')
    cuisines.append(" ".join(temp))  

In [31]:
#delete
len(cuisines)

6465

In [33]:
all_res['recomedation']=cuisines+all_res['reviews_list']

<ipython-input-33-16ef06fd939d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_res['recomedation']=cuisines+all_res['reviews_list']


In [35]:
max_features=20000
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(max_features=max_features)
X=vectorizer.fit_transform(all_res['recomedation'])

In [36]:
from scipy import sparse
x_sparse=sparse.csr_matrix(X)
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(x_sparse)
print(similarity)

[[1.         0.39293191 0.24468036 ... 0.43124954 0.11478789 0.18318179]
 [0.39293191 1.         0.22553508 ... 0.05099347 0.12569921 0.25117473]
 [0.24468036 0.22553508 1.         ... 0.14082663 0.07156918 0.10956207]
 ...
 [0.43124954 0.05099347 0.14082663 ... 1.         0.01440724 0.05201098]
 [0.11478789 0.12569921 0.07156918 ... 0.01440724 1.         0.11314133]
 [0.18318179 0.25117473 0.10956207 ... 0.05201098 0.11314133 1.        ]]


In [37]:
similarity.shape

(6465, 6465)

In [40]:
all_res.set_index('name',inplace=True)

In [43]:
indices = pd.Series(all_res.index)

In [47]:
def recommend(name, cosine_similarities = similarity):
    
    # Create a list to put top restaurants
    recommend_restaurant = []
    
    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]
    
    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extract top 30 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:31].index)
    
    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(all_res.index)[each])
    
    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'Cost'])
    
    # Create the top 30 similar restaurants with some of their columns
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(all_res[['cuisines','Mean Rating', 'Cost']][all_res.index == each].sample()))
    
    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'Cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    
    return df_new
recommend('Pai Vihar')

TOP 10 RESTAURANTS LIKE Pai Vihar WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,Cost
Mudde Madappa Mess,South Indian,4.100000,150
Chaitanya Cafe,"South Indian, North Indian, Fast Food",3.900000,300
Samosa Singh,"Street Food, Fast Food, Rolls, Desserts",3.816667,200
Matru Sagar,"South Indian, North Indian",3.800000,300
Upahar Sagar,"South Indian, Chinese, North Indian",3.800000,350
Filmy Kitchen,"North Indian, South Indian, Italian, Chinese, ...",3.700000,400
Mayura Sagar,"Chinese, North Indian, South Indian",3.600000,250
Parama Ruchi,"South Indian, North Indian, Chinese, Street Food",3.500000,300
Udupi Food Junction,"South Indian, North Indian",3.500000,250
Navya Andhra Style Restaurant,"North Indian, South Indian",3.500000,200
